In [1]:
import torch
import torch.nn as nn
import torch.utils.data as td
import torchvision as tv
import torchvision.transforms as tf
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from ckpt_manager import CheckpointManager
from tqdm import tqdm
import numpy as np

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
torch.cuda.device_count()

1

In [3]:
class ResNetDecBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, upsample=None):
        super().__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(in_channels))
        if stride != 1:
            self.conv2 = nn.Sequential(
                            nn.Upsample(scale_factor=stride),
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(out_channels))
            self.shortcut = nn.Sequential(
                            nn.Upsample(scale_factor=stride),
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1))
        else:
            self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(out_channels))
            self.shortcut = nn.Sequential()
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [4]:
class ResNet18Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.inchannel = 64
        backbone = torchvision.models.resnet18(weights="DEFAULT")

        self.conv1 = nn.Sequential(
            backbone.conv1,
            backbone.bn1,
            backbone.relu,
            backbone.maxpool
        )
        self.layer1 = backbone.layer1
        self.layer2 = backbone.layer2
        self.layer3 = backbone.layer3
        self.layer4 = backbone.layer4
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        return out

In [5]:
class ResNet18Decoder(nn.Module):
    def __init__(self, Block):
        super().__init__()
        self.inchannel = 512
        self.up = nn.Upsample(scale_factor=2)
        self.layer1 = self.make_layer(Block, 256, 2, stride=2)
        self.layer2 = self.make_layer(Block, 128, 2, stride=2)
        self.layer3 = self.make_layer(Block, 64, 2, stride=2)
        self.layer4 = self.make_layer(Block, 64, 2, stride=1)
        self.resize = nn.Sequential(
                            nn.Upsample(scale_factor=4),
                            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1))
        
    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = x.view(-1, 512, 8, 8)
        out = self.up(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.resize(out)
        return out

In [6]:
class ResNetAutoencoder(nn.Module):
    def __init__(self, quantize_factor):
        super().__init__()
        self.enc = ResNet18Encoder()
        self.dec = ResNet18Decoder(ResNetDecBlock)
        self.quantize_factor = quantize_factor
    
    def forward(self, x):
        out = self.enc(x)
        out = F.sigmoid(out)
        out = out + (1 / 2 ** self.quantize_factor) * (torch.rand_like(out) * 0.5 - 0.5)
        out = torch.log(out / (1 - out))
        out = self.dec(out)
        return F.sigmoid(out)

In [7]:
transform = tf.Compose(
        [
         tf.ToTensor()
        ]
)

batch_size = 64

In [8]:
dataset = torchvision.datasets.ImageFolder('universal-dataset', transform=transform)
loader = td.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=8)

In [9]:
model = ResNetAutoencoder(2).cuda()

In [10]:
manager = CheckpointManager(
    assets={
        'model' : model.state_dict()
    },
    directory='training_checkpoints_B=2',
    file_name='ResNetAutoencoder',
    maximum=5,
    file_format='pt'
)

In [11]:
loaded_state = manager.load()
model.load_state_dict(loaded_state['model'])

Loading states from training_checkpoints_B=2/ResNetAutoencoder_132.pt


<All keys matched successfully>

In [14]:
optimizer = optim.Adam(model.parameters(), lr=1e-4 * 0.7)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.7)
criterion = nn.MSELoss()

model.train()

In [13]:
EPOCHS = 1
SAVE_LOG_ITERS = 100
losses = []
for epoch in range(EPOCHS):
    for i, (batch, _) in tqdm(enumerate(loader)):
        batch = batch.cuda()
        
        predicted = model(batch)
        
        loss = criterion(predicted, batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
        if (i + 1) % SAVE_LOG_ITERS == 0:
            print(f"Epoch: {epoch} | iter: {i} | loss: {np.array(losses[-SAVE_LOG_ITERS:]).mean()}")
            manager.save()
    scheduler.step()

In [60]:
torch.save(model.enc.state_dict(), 'model_B=8/encoder.model')
torch.save(model.dec.state_dict(), 'model_B=8/decoder.model')

In [12]:
torch.save(model.enc.state_dict(), 'model_B=2/encoder.model')
torch.save(model.dec.state_dict(), 'model_B=2/decoder.model')